In [0]:
# A many to many RNN model which is used to gain 
# a reasonable weigth matrix for further use.
# Input string is one time step ahead of the target output string.
# Input dim (batch_size, timesteps, num_tokens)

In [2]:
from keras.models import Model, Input
from keras.layers import Concatenate
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import random
import io

Using TensorFlow backend.


In [3]:
# load text
from google.colab import files
uploaded = files.upload()

Saving warpeace_input.txt to warpeace_input.txt


In [4]:
with io.open('warpeace_input.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 3196213


In [0]:
# split text
train_x = text[:1800000]
val_x = text[1800000:]

In [6]:
# number of characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))
# index dict
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [7]:
# build model: 3 layered LSTM with residual connection
input_char = Input(shape=(None, len(chars)))
hidden1 = LSTM(256, return_sequences=True)(input_char)
concat1 = Concatenate(axis=-1)([input_char, hidden1])
hidden2 = LSTM(256, return_sequences=True)(concat1)
concat2 = Concatenate(axis=-1)([input_char, hidden2])
hidden3 = LSTM(256, return_sequences=True)(concat2)
concat3 = Concatenate(axis=-1)([hidden1, hidden2, hidden3])
output = Dense(len(chars), activation="softmax")(concat3)

model = Model(input_char, output)

W0713 17:50:07.815758 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 17:50:07.853733 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 17:50:07.864151 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 57)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, None, 256)    321536      input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 313)    0           input_1[0][0]                    
                                                                 lstm_1[0][0]                     
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, None, 256)    583680      concatenate_1[0][0]              
__________

In [9]:
# cut the text in sequences of maxlen characters to create training data
maxlen = 50
step = 10
prev_sentences = []
next_sentences = []
for i in range(0, len(train_x) - maxlen, step):
    prev_sentences.append(train_x[i: i+maxlen])
    next_sentences.append(train_x[i+1: i+maxlen+1])
print('number sequences:', len(prev_sentences))

number sequences: 179995


In [10]:
print('Vectorization...')
# create one hot vector
x = np.zeros((len(prev_sentences), maxlen, len(chars)))
y = np.zeros((len(next_sentences), maxlen, len(chars)))
for i, sentence in enumerate(prev_sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        
for i, sentence in enumerate(next_sentences):
    for t, char in enumerate(sentence):
        y[i, t, char_indices[char]] = 1

Vectorization...


In [0]:
test_len = 100000
start_index = random.randint(0, len(val_x) - test_len - 1)
test_prev_sentence = val_x[start_index: start_index+test_len]
test_next_sentence = val_x[start_index+1: start_index+test_len+1]


test_x = np.zeros((1, test_len, len(chars)))
test_y = np.zeros((1, test_len, len(chars)))

for t, char in enumerate(test_prev_sentence):
  test_x[0, t, char_indices[char]] = 1

for t, char in enumerate(test_next_sentence):
  test_y[0, t, char_indices[char]] = 1

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0713 17:53:20.137559 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0713 17:53:20.179061 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [0]:
checkpoint_cb = ModelCheckpoint("warpeace_model.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(patience=3)

In [14]:
model.fit(x, y,
          batch_size=128, epochs=20,
          validation_data = (test_x, test_y),
          callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr]
          )

W0713 17:54:15.949304 140089782818688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0713 17:54:17.692794 140089782818688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 179995 samples, validate on 1 samples
Epoch 1/20
179995/179995 [==============================] - 353s 2ms/step - loss: 1.8355 - acc: 0.4578 - val_loss: 1.5214 - val_acc: 0.5453
Epoch 2/20
179995/179995 [==============================] - 342s 2ms/step - loss: 1.3541 - acc: 0.5831 - val_loss: 1.3969 - val_acc: 0.5817
Epoch 3/20
179995/179995 [==============================] - 336s 2ms/step - loss: 1.2407 - acc: 0.6124 - val_loss: 1.3573 - val_acc: 0.5953
Epoch 4/20
179995/179995 [==============================] - 334s 2ms/step - loss: 1.1784 - acc: 0.6292 - val_loss: 1.3472 - val_acc: 0.6020
Epoch 5/20
179995/179995 [==============================] - 334s 2ms/step - loss: 1.1321 - acc: 0.6418 - val_loss: 1.3577 - val_acc: 0.6022
Epoch 6/20
179995/179995 [==============================] - 332s 2ms/step - loss: 1.0939 - acc: 0.6526 - val_loss: 1.3718 - val_acc: 0.6019
Epoch 7/20
179995/179995 [==============================] - 333s 2ms/step - loss: 1.0605 - acc: 0.6625 - val_loss

In [0]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='plot.png', show_shapes=True, show_layer_names=True)

In [17]:
# save to google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [19]:
!ls drive

'Colab Notebooks'   dissertation  'warpeace -res256'   warpeace_text_generation


In [20]:
model_json = model.to_json()
with open("drive/dissertation/War&peace_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("drive/dissertation/War&peace_model_weights.h5")
print("Saved model to drive")

Saved model to drive
